In [2]:
import cv2
import numpy as np


TEMP_TUNER = 1.80
TEMP_TOLERENCE = 70.6


def process_frame(frame):
    
    frame = ~frame
    heatmap = cv2.applyColorMap(frame, cv2.COLORMAP_HOT)
    
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    ret, binary_thresh = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    image_erosion = cv2.erode(binary_thresh, kernel, iterations=1)
    image_opening = cv2.dilate(image_erosion, kernel, iterations=1)
    
    # Get contours from the image obtained by opening operation
    contours, _ = cv2.findContours(image_opening, 1, 2)

    image_with_rectangles = np.copy(heatmap)


    for contour in contours:
        # rectangle over each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        if (w) * (h) < 2400:
            continue

        # Mask is boolean type of matrix.
        mask = np.zeros_like(heatmap_gray)
        cv2.drawContours(mask, contour, -1, 255, -1)

        # Mean of only those pixels which are in blocks and not the whole rectangle selected
        mean = convert_to_temperature(cv2.mean(heatmap_gray, mask=mask)[0])

        # Colors for rectangles and textmin_area
        temperature = round(mean, 2)
        color = (0, 255, 0) if temperature < 70.6 else (
            255, 255, 127)
        

        # Draw rectangles for visualisation
        image_with_rectangles = cv2.rectangle(image_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image_with_rectangles, "{} C".format(temperature), (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
    
    return image_with_rectangles


def whole_frame():
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()

        if ret == True:
            frame = process_frame(frame)

            cv2.imshow('Thermal', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        else:
            break

    cap.release()
    cv2.destroyAllWindows()


def process_face(frame):
    
    frame = ~frame
    heatmap = cv2.applyColorMap(frame,cv2.COLORMAP_HOT)
    
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    ret, binary_thresh = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    image_erosion = cv2.erode(binary_thresh, kernel, iterations=1)
    image_opening = cv2.dilate(image_erosion, kernel, iterations=1)
    

    image_with_rectangles = np.copy(heatmap)
    
    return image_with_rectangles



def convert_to_temperature(pixel_avg):
    """
    Converts pixel value (mean) to temperature depending upon the camera hardware
    """
    f = pixel_avg / TEMP_TUNER
    c = (f - 32) * 5/9

    
    return f

    


def only_face():
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

    while(cap.isOpened()):
        ret, frame = cap.read()
        frame = cv2.flip(frame, 180)

        if ret == True:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            output = process_face(frame)

            for (x,y,w,h) in faces:

                roi = output[y:y+h, x:x+w]
                print(roi)
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

                 # Mask is boolean type of matrix.
                mask = np.zeros_like(roi_gray)

                # Mean of only those pixels which are in blocks and not the whole rectangle selected
                mean = convert_to_temperature(np.mean(roi_gray))

                # Colors for rectangles and textmin_area
                temperature = round(mean, 2)
                color = (0, 255, 0) if temperature < TEMP_TOLERENCE else (
                    255, 255, 255)
                

                # Draw rectangles for visualisation
                output = cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(output, "{} C".format(temperature), (x, y-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
                if temperature > 100:
                    print("image captured")
                    cv2.imwrite(filename='saved_img.jpg', img=frame)
                    '''cap.release()
                    img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
                    img_new = cv2.imshow("Captured Image", img_new)
                    cv2.waitKey(1650)
                    cv2.destroyAllWindows()
                    print("Processing image...")
                    img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
                    print("Converting RGB image to grayscale...")
                    gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
                    print("Converted RGB image to grayscale...")
                    print("Resizing image to 28x28 scale...")
                    print("Resized...")
                    img_resized = cv2.imwrite(filename='saved_img-final.jpg', img=gray)'''

                
            cv2.imshow('Thermal', output)
            # out.write(output)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        else:
            break

    cap.release()
    # out.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # whole_frame()
    only_face()

[[[  0 115 255]
  [  0 145 255]
  [  0 158 255]
  ...
  [160 255 255]
  [150 255 255]
  [165 255 255]]

 [[  0 125 255]
  [  0 162 255]
  [  0 182 255]
  ...
  [170 255 255]
  [165 255 255]
  [165 255 255]]

 [[  0 118 255]
  [  0 160 255]
  [  0 190 255]
  ...
  [175 255 255]
  [175 255 255]
  [165 255 255]]

 ...

 [[  0   8 255]
  [  0  15 255]
  [  0  22 255]
  ...
  [  0 148 255]
  [  0  70 255]
  [  0  25 255]]

 [[  0  13 255]
  [  0  13 255]
  [  0  20 255]
  ...
  [  0 225 255]
  [  0 210 255]
  [  0 165 255]]

 [[  0  15 255]
  [  0  13 255]
  [  0  18 255]
  ...
  [  0 225 255]
  [  0 242 255]
  [  0 228 255]]]
[[[  0 218 255]
  [  0 208 255]
  [  0 195 255]
  ...
  [155 255 255]
  [155 255 255]
  [150 255 255]]

 [[  0 162 255]
  [  0 188 255]
  [  0 170 255]
  ...
  [160 255 255]
  [155 255 255]
  [145 255 255]]

 [[  0 130 255]
  [  0 130 255]
  [  0 168 255]
  ...
  [160 255 255]
  [150 255 255]
  [145 255 255]]

 ...

 [[  0  10 255]
  [  0  13 255]
  [  0   8 255]
  ..

In [14]:
a = 0
b = 255
x = 90
xminimum = 80
ranges = 110-80
xnormalized = a + (((x - xminimum) * (b - a)) / ranges)
print(xnormalized)

85.0


In [17]:
xnormalized = 255
x = (((xnormalized-a)*ranges)/(b-a))+xminimum
print(x)


110.0
